In [26]:

#import modules
import math
import sys


#arguments
"""txtfile=sys.argv[1]
userName=sys.argv[2]
n=int(sys.argv[3])
k=int(sys.argv[4])"""

#arguments
txtfile="ratings-dataset.tsv"


Eternal Sunshine of the Spotless Mind 4.19946
Kill Bill: Vol. 2 4.19182
Kill Bill: Vol. 1 4.17114
Erin Brockovich 4.16667
Sin City 4.16667


In [37]:
userName=14756
n=20
k=2

#global variables
uIndex=0
urIndex=1
movieIndex=2
Print=False


#define functions
def setPrint():
    global Print
    Print=True

def resetPrint():
    global Print
    Print=False


def getUserRatings(myList):
    tempDict={}
    for eachList in myList:
        movie=eachList[movieIndex]
        user=eachList[uIndex]
        userRating=eachList[urIndex]
        if movie in tempDict:
            tempDict[movie][user]=float(userRating)
        else:
            #tempDict[movie][user]=userRating
            tempDict[movie]={}
            tempDict[movie][user]=float(userRating)
        
    return tempDict

def getItemsList(myList):
    tempList=[]
    for eachList in myList:
        movie=eachList[movieIndex]        
        if movie not in tempList:
            tempList.append(movie)
    tempList.sort()
    return tempList

def getUnratedMovies(user,iList,mDict):
    tempDict={}
    tempDict[user]=[]
    userList=[]
    for eachMovie in mDict:        
        if user not in mDict[eachMovie]:            
            tempDict[user].append(eachMovie)
    tempDict[user].sort()
    return tempDict

def getCommonUsers(dict1,dict2):
    tempList=[]
    for key in dict1:
        if key in dict2:
           tempList.append(key)    
    return tempList

def getAvg(myList,myDict):
    total=0
    if len(myList)==0:
        return 0
    for user in myList:
        total+=myDict[user]
        
    mean=float(total)/float(len(myList))        
    return mean
        

In [ ]:
#Main Function
if __name__=="__main__":
    ratingsFile=open(txtfile,"r+") #Read the .tsv and create a list of lists for each line
    lines=ratingsFile.read()
    lines=lines.split('\n')
    linesList=[]
    for eachItem in lines:
        eachItem=eachItem.split('\t')
        if eachItem!=['']:
            linesList.append(eachItem)


#Create a Dictionary in the format - 
#moviesDict{
#	movie1:{
#		user1:Rating1,
#		user2:Rating2....}
#	movie2:{
#		user1:Rating1,
#		user2:Rating2....}
    moviesDict=getUserRatings(linesList) 
    if Print==True:
        print(moviesDict)


    unratedMoviesDict={}
    itemsList=getItemsList(linesList) #List of all the unique movies in the given .tsv
#Dictionary of all the unrated movies by a user(passed as argv)
#unratedMoviesDict={User:[movie1,movie2...]}
    unratedMoviesDict=getUnratedMovies(userName,itemsList,moviesDict) 

In [38]:
#Calculating the pearson correlation
W={}
for movie in unratedMoviesDict[userName]:
    W[movie]={}
for urMovie in W:
    for movie in moviesDict:
        if movie!=urMovie:
            commonUsersList=getCommonUsers(moviesDict[urMovie],moviesDict[movie]) #Get the list of co-rated users for unrated movie, rated movie pair              
            avgURMovie=getAvg(commonUsersList,moviesDict[urMovie]) #Average rating of unrated movie
            avgRMovie=getAvg(commonUsersList,moviesDict[movie]) #Average rating of rated movie
            Nr=Dr=term1=term2=0

In [ ]:
#Nr=summation of [Ru,i - Avg(R)] * [Ru,j - Avg(R)]
#term1 = Square root of summation of [Ru,i - Avg(R)]^2
#term1 = Square root of summation of [Ru,j - Avg(R)]^2
#Dr=term1 * term2
#Pearson correlation= Nr/Dr
                for user in commonUsersList:
                    Nr+=(moviesDict[urMovie][user]-avgURMovie)* (moviesDict[movie][user]-avgRMovie) #Nr=Numerator
                    term1+=(moviesDict[urMovie][user]-avgURMovie)**2                    
                    term2+=(moviesDict[movie][user]-avgRMovie)**2
                term1=math.sqrt(term1)
                term2=math.sqrt(term2)
                Dr=term1*term2 #Dr=Denominator
                if Dr!=0:
                    val=Nr/Dr
                else:
                    val=0
                W[urMovie][movie]=val
    if Print==True:
        print("W")

    
    newDict={}
    for urMovie in W:
        tempList=[]        
        d=W[urMovie]
#Sort all the values in dictionary in descending order. If the values are same, then sort ther keys in alphabetical order
        #tempList=[v[0] for v in sorted(d.iteritems(), key=lambda(k, v): (-v, k))]
        tempList=[v[0] for v in sorted(iter(d.items()), key=lambda k_v: (-k_v[1], k_v[0]))]
        newDict[urMovie]=tempList
    
    if Print==True:
        print("NewDict")


#Make prediction for movies with highest pearson correlation for the given user		
    predictionDict={}    
    for urMovie in newDict:
        Nr=Dr=0.00000
        m=n        
        if len(newDict[urMovie]) < n:
            m=len(newDict[urMovie])        
        i=0
        while i<m:
            rMovie=newDict[urMovie][i]
            if userName in moviesDict[rMovie]:
#Nr=Summation of Ru,n * Wi,n
#Dr=Summation of Wi,n
#Prediction for user u for item i = Nr/Dr
                Nr+=W[urMovie][rMovie]*moviesDict[rMovie][userName]
                Dr+=W[urMovie][rMovie]
            else:
                m=m+1
            i+=1
        if Dr!=0:   
            predictionDict[urMovie]=float(Nr)/float(Dr)
        else:
            predictionDict[urMovie]=0.0

#Sort all the values in dictionary in descending order. If the values are same, then sort ther keys in alphabetical order            
    tempList=sorted(list(predictionDict.items()),key=lambda k_v:(-k_v[1],k_v[0]))    
    for i in range(k):
        movie=tempList[i]        
        print("%s %.5f"%(movie[0],movie[1]))   
    
       
#close all files  
    #ratingsFile.close()

In [25]:
predictionDict

{'A Beautiful Mind': 3.9845825841059526,
 'Ace Ventura: Pet Detective': 3.8338896316329145,
 'Almost Famous': 4.033333333333333,
 'AmAlie': 3.9562714557329794,
 'American Beauty': 0.0,
 'Apollo 13': 4.060843677159905,
 'Batman Forever': 0.0,
 'Beauty and the Beast': 4.096031443238852,
 'Braveheart': 4.145337685604071,
 'Cast Away': 4.029345764290019,
 "Charlie's Angels": 3.95083895089428,
 'Chicken Run': 3.8365537797606195,
 'Clear and Present Danger': 3.8125,
 'Dances with Wolves': 4.163236283094399,
 'Die Hard: With a Vengeance': 4.012881474421681,
 'Donnie Darko': 3.6977466716803185,
 'Dumb and Dumber': 3.9256155298805897,
 'E.T. the Extra-Terrestrial': 4.059867106666399,
 'Erin Brockovich': 4.166666666666667,
 'Eternal Sunshine of the Spotless Mind': 4.199458439316914,
 'Fargo': 4.038659169967981,
 'Fight Club': 3.998069144585473,
 'Forrest Gump': 3.8004417006456483,
 'Gladiator': 3.9597938741713388,
 'Harry Potter and the Chamber of Secrets': 3.820669870460446,
 "Harry Potter and 

In [ ]:
#Main Function
if __name__=="__main__":
    txtfile = "ratings-dataset.tsv"
    ratingsFile=open(txtfile,"r+") #Read the .tsv and create a list of lists for each line
    lines=ratingsFile.read()
    lines=lines.split('\n')
    linesList=[]
    for eachItem in lines:
        eachItem=eachItem.split('\t')
        if eachItem!=['']:
            linesList.append(eachItem)

In [32]:
#Create a Dictionary in the format - 
#moviesDict{
#	movie1:{
#		user1:Rating1,
#		user2:Rating2....}
#	movie2:{
#		user1:Rating1,
#		user2:Rating2....}
moviesDict=getUserRatings(training_list) 
print(moviesDict)

{1: {1488844: 3.0, 822109: 5.0, 885013: 4.0, 823519: 3.0, 893988: 3.0, 124105: 4.0, 1248029: 3.0, 1842128: 4.0, 2238063: 3.0, 1503895: 4.0, 2207774: 5.0, 2590061: 3.0, 2442: 3.0, 543865: 4.0, 1209119: 4.0, 804919: 4.0, 1086807: 3.0, 1711859: 4.0, 372233: 5.0, 1080361: 3.0, 1245640: 3.0, 558634: 4.0, 2165002: 4.0, 1181550: 3.0, 1227322: 4.0, 427928: 4.0, 814701: 5.0, 808731: 4.0, 662870: 5.0, 337541: 5.0, 786312: 3.0, 1133214: 4.0, 1537427: 4.0, 1209954: 5.0, 2381599: 3.0, 525356: 2.0, 1910569: 4.0, 2263586: 4.0, 2421815: 2.0, 1009622: 1.0, 1481961: 2.0, 401047: 4.0, 2179073: 3.0, 1434636: 3.0, 93986: 5.0, 1308744: 5.0, 1905581: 5.0, 2508819: 3.0, 1578279: 1.0, 1159695: 4.0, 2588432: 3.0, 2423091: 3.0, 470232: 4.0, 2148699: 2.0, 1342007: 3.0, 466135: 4.0, 2472440: 3.0, 1927580: 4.0, 716874: 5.0, 4326: 4.0, 1546549: 5.0, 1493697: 1.0, 880166: 5.0, 535396: 2.0, 494609: 4.0, 1961619: 5.0, 883478: 4.0, 793564: 4.0, 1567202: 2.0, 573537: 4.0, 1972040: 4.0, 1838912: 3.0, 411705: 4.0, 2244518:

In [14]:
%store -r training_list 

In [31]:
#arguments
#txtfile=sys.argv[1]
userName='Kluver'
#n=int(sys.argv[3])
#k=int(sys.argv[4])

unratedMoviesDict={}
itemsList=getItemsList(linesList) #List of all the unique movies in the given .tsv
#Dictionary of all the unrated movies by a user(passed as argv)
#unratedMoviesDict={User:[movie1,movie2...]}
unratedMoviesDict=getUnratedMovies(userName,itemsList,moviesDict) 

Stored 'unratedMoviesDict' (dict)


In [33]:
unratedMoviesDict={14756: [7, 1]}

In [10]:
def getUserRatings(myList):
    tempDict={}
    for eachList in myList:
        movie=eachList[movieIndex]
        user=eachList[uIndex]
        userRating=eachList[urIndex]
        if movie in tempDict:
            tempDict[movie][user]=float(userRating)
        else:
            #tempDict[movie][user]=userRating
            tempDict[movie]={}
            tempDict[movie][user]=float(userRating)
        
    return tempDict

def getItemsList(myList):
    tempList=[]
    for eachList in myList:
        movie=eachList[movieIndex]        
        if movie not in tempList:
            tempList.append(movie)
    tempList.sort()
    return tempList

def getUnratedMovies(user,iList,mDict):
    tempDict={}
    tempDict[user]=[]
    userList=[]
    for eachMovie in mDict:        
        if user not in mDict[eachMovie]:            
            tempDict[user].append(eachMovie)
    tempDict[user].sort()
    return tempDict

def getCommonUsers(dict1,dict2):
    tempList=[]
    for key in dict1:
        if key in dict2:
           tempList.append(key)    
    return tempList

def getAvg(myList,myDict):
    total=0
    if len(myList)==0:
        return 0
    for user in myList:
        total+=myDict[user]
        
    mean=float(total)/float(len(myList))        
    return mean
        
        
#global variables
uIndex=0
urIndex=1
movieIndex=2
Print=False